# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
df = pd.read_csv('main_task_new.csv')

In [3]:
len( df['City'].unique() )
df['Price Range'].unique()
df.info()
df['Price Range'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [4]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [5]:
# Обработка Price Range - приводим к числовому виду, заполняем пропуски, создаем столбец маркирующий пропуски
def normalizePriceRange(price_range):
    if(price_range == '$'):
        return 1
    
    if(price_range == '$$ - $$$'):
        return 2
    
    if(price_range == '$$$$'):
        return 3
    
    
    return 2

def emptyMarker(price_range):
    if(pd.isnull(price_range)):
        return 1
    return 0

df['price_range'] = df['Price Range'].apply(normalizePriceRange)
# df['empty_price_range'] = df['Price Range'].apply(emptyMarker)

In [6]:
# Обработка городов, вычисление столбца с рейтингом по городу (вычисляется долго, надо подождать)

cities = df['City'].unique()
min_max_rankings = {}
for city in cities:
    min_ranking = df.loc[df['City'] == city, 'Ranking'].min()
    max_ranking = df.loc[df['City'] == city, 'Ranking'].max()
    item = {};
    item['min'] = min_ranking
    item['max'] = max_ranking
    min_max_rankings[city] = item

ds = pd.Series([], dtype='float') 
for index, row in df.iterrows():
    min_ranking = min_max_rankings[ row['City'] ]['min']
    max_ranking = min_max_rankings[ row['City'] ]['max']
    
    if(max_ranking - min_ranking == 0):
        ds.at[index] = 0
        continue
    
    ranking_by_city = (row['Ranking'] - min_ranking) / (max_ranking - min_ranking)
    ds.at[index] = ranking_by_city

df['ranking_by_city'] = ds;    

In [7]:
# Получаем данные о кухнях
cuisine_list = {}
for raw_string in df['Cuisine Style']:
    if(isinstance(raw_string, str) != True):
        continue
    cleared_string = raw_string.replace("'", '').replace('"', '').replace(']', '').replace('[', '').replace(' ', '')
    items = cleared_string.split(',')
    
    for item in items:
        if( isinstance(cuisine_list.get(item), int) != True ):
            cuisine_list[item] = 0
        cuisine_list[item] = cuisine_list[item] + 1
        

In [8]:
# Добавляем столбец с числом кухонь
def countCuisineStyle(cuisine_style):
    if(isinstance(cuisine_style, str) != True):
        return 1
    cleared_string = cuisine_style.replace("'", '').replace('"', '').replace(']', '').replace('[', '').replace(' ', '')
    return len( cleared_string.split(',') )

cuisine_style_count = df['Cuisine Style'].apply(countCuisineStyle)
df['cousine_style_count'] = cuisine_style_count

In [9]:
# Добавляем столбцы с кухнями
def addCuisine(raw_string, c_title):
    if(isinstance(raw_string, str) != True):
        return 0
    cleared_string = raw_string.replace("'", '').replace('"', '').replace(']', '').replace('[', '').replace(' ', '')
    items = cleared_string.split(',')
    
    for item in items:
        if item == c_title:
            return 1
        
    return 0

for c_title in cuisine_list.keys():
    df[c_title] = df['Cuisine Style'].apply(addCuisine, args=(c_title,))

In [10]:
# Чистка числовых столбцов
df['Ranking'].fillna(0)
df['Rating'].fillna(0)
df['Number of Reviews'].fillna(0)

def clearFields(item):
    if(isinstance(item, str) == True or pd.isnull(item)):
        return 0

    return item

df['Ranking'] = df['Ranking'].apply(clearFields)
df['Rating'] = df['Rating'].apply(clearFields)
df['Number of Reviews'] = df['Number of Reviews'].apply(clearFields)

In [11]:
# Обработка отзывов - вычисляем частоту отзывов
def reviewsFreq(string):
    if(isinstance(string, str) != True):
        return -1

    m = re.findall('\d\d/\d\d/\d\d\d\d', string)
    
    if(len(m) < 2):
        return -1
    
    date_time1 = datetime.datetime.strptime(m[0], '%m/%d/%Y')
    date_time2 = datetime.datetime.strptime(m[1], '%m/%d/%Y')
    
    if(date_time1 == date_time2):
        return 0
    
    days = int( str(date_time1 - date_time2).split()[0] );
    return abs(days)

reviews_freq = df['Reviews'].apply(reviewsFreq)

# Заполняем пропуски средним значением
mean = int(reviews_freq.mean())
i = 0
for freq in reviews_freq:
    if(freq == -1):
        reviews_freq[i] = mean
    i = i + 1

df['reviews_freq'] = reviews_freq

In [12]:
df = df.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [13]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [14]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [15]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [16]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [17]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [18]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2190595
